In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv('ML_Datasets/HP_train.csv')
df_test = pd.read_csv('ML_Datasets/HP_test.csv')

In [3]:
columns_numeric = list(df_train.select_dtypes(exclude='object').columns)
columns_numeric.remove('SalePrice')

In [4]:
columns_cat = list(df_train.select_dtypes(include='object').columns)

# Missing Data => SimpleImputer Class

In [5]:
from sklearn.impute import SimpleImputer

In [6]:
sum(df_train.isna().sum()>0)

19

In [7]:
imputer_numeric = SimpleImputer(strategy='median')
imputer_numeric.fit(df_train[columns_numeric]) # finding out median of numeric columns
df_train[columns_numeric] = imputer_numeric.transform(df_train[columns_numeric])

In [8]:
imputer_cat = SimpleImputer(strategy='most_frequent')
imputer_cat.fit(df_train[columns_cat]) # finding out median of numeric columns
df_train[columns_cat] = imputer_cat.transform(df_train[columns_cat])

In [9]:
sum(df_train.isna().sum()>0)

0

# Scaling->Numeric

In [10]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
scaler = MinMaxScaler()
scaler.fit(df_train[columns_numeric])
df_train[columns_numeric] = scaler.transform(df_train[columns_numeric])

# One Hot Encoding->categorical

In [12]:
from sklearn.preprocessing import OneHotEncoder

In [13]:
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
ohe.fit(df_train[columns_cat])

OneHotEncoder(handle_unknown='ignore', sparse=False)

In [14]:
len(ohe.get_feature_names())

252

In [15]:
ohe.transform(df_train[columns_cat])

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [16]:
df_train[ohe.get_feature_names()] = ohe.transform(df_train[columns_cat])

C:\Users\Doted\anaconda3\lib\site-packages\pandas\core\frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


In [17]:
len(df_train.columns)

333

# Building ML model

In [18]:
# numeric + encoded
X_train = df_train[columns_numeric + list(ohe.get_feature_names())]
y_train = df_train['SalePrice']

In [19]:
from sklearn.ensemble import RandomForestRegressor

In [20]:
model = RandomForestRegressor()
model.fit(X_train,y_train)

RandomForestRegressor()

# Make Predictions -> test

In [21]:
df_test[columns_numeric] = imputer_numeric.transform(df_test[columns_numeric])
df_test[columns_cat] = imputer_cat.transform(df_test[columns_cat])

In [22]:
df_test[columns_numeric] = scaler.transform(df_test[columns_numeric])

In [23]:
df_test[ohe.get_feature_names()] = ohe.transform(df_test[columns_cat])

C:\Users\Doted\anaconda3\lib\site-packages\pandas\core\frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


In [24]:
X_test = df_test[columns_numeric + list(ohe.get_feature_names())]

In [25]:
yp = model.predict(X_test)

# Create submission file

In [26]:
df_test = pd.read_csv('ML_Datasets/HP_test.csv')

In [27]:
df_test['SalePrice'] = yp

In [28]:
df_test[['Id','SalePrice']].to_csv('sub1.csv',index = False)